<a href="https://colab.research.google.com/github/falawar7/AAI_635O/blob/main/Week2/Exercise_Fe_Collaborative_Filtering_(Chapter_4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [3]:
df = pd.read_csv('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/xyz.csv',encoding= 'unicode_escape')

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
df.isnull().sum().sort_values(ascending=False)

,0
CustomerID,135080
Description,1454
StockCode,0
InvoiceNo,0
Quantity,0
InvoiceDate,0
UnitPrice,0
Country,0


In [6]:
df_new = df.dropna()

In [7]:
df_new.describe()

# Here we can see quantity has some negative values which is a part of incorrect data so we will drop such entries

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [8]:
df_new = df_new[df_new.Quantity > 0]

In [9]:
df_new.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


## User-to-User Collaborative Filtering

In [10]:
# We are creating a df which contains CustomerID and whether they have ever purchased a product using groupby

purchase = (df_new.groupby(['CustomerID', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))

In [11]:
purchase.head(30)

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12352.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12354.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12355.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#We are getting the quantity ordered (example : 48,24,126) while we just want to know if that particular item is purchased or not
#thus we are encoding units as 1(if purchased) or 0(not purchased)

def encode_units(x):
    if x < 1:
        return 0
    if x >= 1:
        return 1


purchase = purchase.applymap(encode_units)

<ipython-input-12-6797b74d7b9b>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  purchase = purchase.applymap(encode_units)


In [13]:
purchase.head(30)

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12352.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12353.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12354.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12355.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
user_similarity = cosine_similarity(purchase)

In [16]:
user_similarity_df = pd.DataFrame(user_similarity,index=purchase.index,columns=purchase.index)

In [17]:
user_similarity_df

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.012938,0.136641,0.094742,...,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.101565,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.168053,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.121900,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.113547,0.015237
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044118,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.062318,0.000000,0.000000,0.000000,0.000000,0.0,0.041523,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.105409,1.000000,0.119523,0.000000,0.000000,0.000000
18281.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.049629,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.119523,1.000000,0.000000,0.045835,0.000000
18282.0,0.0,0.113776,0.000000,0.067574,0.000000,0.037582,0.0,0.000000,0.160128,0.079305,...,0.0,0.174078,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.017504,0.000000


In [18]:
def similar_users(user_id,k=5):
    # separating df rows for the entered user id
    user = user_similarity_df[user_similarity_df.index == user_id]

    # a df of all other users
    other_users = user_similarity_df[user_similarity_df.index != user_id]

    # calc cosine similarity between user and each other user
    similarities = cosine_similarity(user,other_users)[0].tolist()

    # create list of indices of these users
    indices = other_users.index.tolist()

    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))

    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(),reverse=True)

    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]

    return users

In [22]:
simu = similar_users(12347)

simu
# further the similar users can be stored in a list and later we can display the items purchased by the similar users

[18287.0, 18283.0, 18282.0, 18281.0, 18280.0]

In [23]:
def simu_recommendation(userid):

    simu = similar_users(userid)

    #obtaining all the items bought by similar users
    simu_rec = []
    for j in simu:
        desc = df_new[df_new["CustomerID"]==j]['Description'].to_list()
        simu_rec.append(desc)

    #this gives us multi-dimensional list
    # we need to flatten it
    flat_list = []
    for sublist in simu_rec:
        for item in sublist:
            flat_list.append(item)
    final_list = list(dict.fromkeys(flat_list))

    # storing 10 random recommendations in a list
    ten_recs = random.sample(final_list, 10)

    print('Items bought by Similar users based on Cosine Similarity')

    #returning 10 random recommendations
    return ten_recs

In [24]:
simu_recommendation(12347)

Items bought by Similar users based on Cosine Similarity


['PACK OF 60 DINOSAUR CAKE CASES',
 'LARGE PURPLE BABUSHKA NOTEBOOK  ',
 'SET/6 PURPLE BUTTERFLY T-LIGHTS',
 'LUNCH BAG RED VINTAGE DOILY',
 'RED RETROSPOT CHARLOTTE BAG',
 'PLASTERS IN TIN SPACEBOY',
 'BLUE CIRCLES DESIGN MONKEY DOLL',
 'BOX OF 6 MINI VINTAGE CRACKERS',
 'APPLE BATH SPONGE',
 'CHILDRENS APRON SPACEBOY DESIGN']

## User-to-User Collaborative using KNN

In [25]:
# For passing our sparse matrix into KNN we need to convert it into CSR
# CSR divides a sparse matrix into 3 arrays : values, extent of rows, index of columns

from scipy.sparse import csr_matrix

purchase_matrix = csr_matrix(purchase.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(purchase_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [26]:
simu_knn = []

In [27]:
def similar_users_knn(purchase,query_index):
    distances, indices = model_knn.kneighbors(purchase.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(purchase.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, purchase.index[indices.flatten()[i]], distances.flatten()[i]))
            simu_knn.append(purchase.index[indices.flatten()[i]])

In [28]:
similar_users_knn(purchase,1497)

Recommendations for 14389.0:

1: 16748.0, with distance of 0.47513611891852214:
2: 15417.0, with distance of 0.5065362287801733:
3: 14489.0, with distance of 0.5232687053772038:
4: 17031.0, with distance of 0.5607023148930206:
5: 15747.0, with distance of 0.5695947101270704:


In [29]:
simu_knn

[np.float64(16748.0),
 np.float64(15417.0),
 np.float64(14489.0),
 np.float64(17031.0),
 np.float64(15747.0)]

In [30]:
def simu_recommendation_knn(simu_knn):


    #obtaining all the items bought by similar users
    simu_rec = []
    for j in simu_knn:
        desc = df_new[df_new["CustomerID"]==j]['Description'].to_list()
        simu_rec.append(desc)

    #this gives us multi-dimensional list
    # we need to flatten it
    flat_list = []
    for sublist in simu_rec:
        for item in sublist:
            flat_list.append(item)
    final_list = list(dict.fromkeys(flat_list))

    # storing 10 random recommendations in a list
    ten_recs = random.sample(final_list, 10)

    print('Items bought by Similar users based on KNN')

    #returning 10 random recommendations
    return ten_recs

In [31]:
simu_recommendation_knn(simu_knn)

Items bought by Similar users based on KNN


['LARGE DECO JEWELLERY STAND',
 'JUMBO BAG ALPHABET',
 'LUNCH BAG VINTAGE LEAF DESIGN',
 '3 DRAWER ANTIQUE WHITE WOOD CABINET',
 'JUMBO BAG PAISLEY PARK',
 'EAU DE NILE HEART SHAPE PHOTO FRAME',
 'JUMBO BAG PINK POLKADOT',
 'CHILDRENS APRON APPLES DESIGN',
 'CHARLOTTE BAG PINK POLKADOT',
 'JUMBO BAG VINTAGE CHRISTMAS ']

## Item-to-Item Collaborative Filtering

In [32]:
# We are creating a df which contains item names and whether they have been ever purchased by a customer using groupby

items_purchase = (df_new.groupby(['Description','CustomerID'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Description'))

In [33]:
items_purchase.head(30)

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
Description,,,,,,,,,,,,,,,,,,,,,
4 PURPLE FLOCK DINNER CANDLES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50'S CHRISTMAS GIFT BAG LARGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DOLLY GIRL BEAKER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I LOVE LONDON MINI BACKPACK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I LOVE LONDON MINI RUCKSACK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NINE DRAWER OFFICE TIDY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OVAL WALL MIRROR DIAMANTE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RED SPOT GIFT BAG LARGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
SET 2 TEA TOWELS I LOVE LONDON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
items_purchase = items_purchase.applymap(encode_units)

<ipython-input-34-76e67928d4ee>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  items_purchase = items_purchase.applymap(encode_units)


In [35]:
item_similarity = cosine_similarity(items_purchase)

In [36]:
item_similarity_df = pd.DataFrame(item_similarity,index=items_purchase.index,columns=items_purchase.index)

In [37]:
item_similarity_df.head(10)

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
Description,,,,,,,,,,,,,,,,,,,,,
4 PURPLE FLOCK DINNER CANDLES,1.000000,0.000000,0.017961,0.023583,0.000000,0.000000,0.028050,0.000000,0.031384,0.017125,...,0.0,0.042333,0.043885,0.032001,0.00000,0.026774,0.000000,0.061379,0.000000,0.042333
50'S CHRISTMAS GIFT BAG LARGE,0.000000,1.000000,0.058277,0.038261,0.000000,0.036073,0.060676,0.332508,0.033945,0.083348,...,0.0,0.045787,0.047465,0.034612,0.00000,0.094114,0.000000,0.033193,0.000000,0.022893
DOLLY GIRL BEAKER,0.017961,0.058277,1.000000,0.144437,0.100000,0.037139,0.046852,0.066259,0.061159,0.200227,...,0.0,0.023570,0.048868,0.089087,0.00000,0.096896,0.000000,0.034174,0.028868,0.070711
I LOVE LONDON MINI BACKPACK,0.023583,0.038261,0.144437,1.000000,0.131306,0.048766,0.041013,0.043501,0.126195,0.112676,...,0.0,0.061898,0.048125,0.035093,0.00000,0.039148,0.000000,0.056091,0.000000,0.061898
I LOVE LONDON MINI RUCKSACK,0.000000,0.000000,0.100000,0.131306,1.000000,0.000000,0.000000,0.000000,0.000000,0.095346,...,0.0,0.000000,0.000000,0.089087,0.00000,0.074536,0.000000,0.085436,0.000000,0.000000
NINE DRAWER OFFICE TIDY,0.000000,0.036073,0.037139,0.048766,0.000000,1.000000,0.058001,0.020507,0.032449,0.035411,...,0.0,0.000000,0.045373,0.033086,0.00000,0.041523,0.061898,0.031730,0.000000,0.043769
OVAL WALL MIRROR DIAMANTE,0.028050,0.060676,0.046852,0.041013,0.000000,0.058001,1.000000,0.068986,0.027290,0.059562,...,0.0,0.036811,0.057239,0.069565,0.00000,0.081484,0.052058,0.053371,0.045083,0.110432
RED SPOT GIFT BAG LARGE,0.000000,0.332508,0.066259,0.043501,0.000000,0.020507,0.068986,1.000000,0.057891,0.094763,...,0.0,0.026029,0.053965,0.019676,0.00000,0.049386,0.000000,0.047174,0.000000,0.052058
SET 2 TEA TOWELS I LOVE LONDON,0.031384,0.033945,0.061159,0.126195,0.000000,0.032449,0.027290,0.057891,1.000000,0.041652,...,0.0,0.041187,0.053370,0.046701,0.00000,0.052098,0.029123,0.014929,0.000000,0.020593


In [43]:
def similar_items(item,k=10):
    # separating df rows of the selected item
    item = item_similarity_df[item_similarity_df.index == item]

    # a df of all other items
    other_items = item_similarity_df

    # calc cosine similarity between selected item with other items
    similarities = cosine_similarity(item,other_items)[0].tolist()

    # create list of indices of these items
    indices = other_items.index.tolist()

    # create key/values pairs of item index and their similarity
    index_similarity = dict(zip(indices, similarities))

    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items())

    # grab k items from the top
    top_item_similarities = index_similarity_sorted[:k]
    items = [u[0] for u in top_item_similarities]

    print('Similar items based on purchase behaviour (item-to-item collaborative filtering)')
    return items

In [39]:
similar_items(' 4 PURPLE FLOCK DINNER CANDLES')

Similar items based on purchase behaviour (item-to-item collaborative filtering)


[' 4 PURPLE FLOCK DINNER CANDLES',
 " 50'S CHRISTMAS GIFT BAG LARGE",
 ' DOLLY GIRL BEAKER',
 ' I LOVE LONDON MINI BACKPACK',
 ' I LOVE LONDON MINI RUCKSACK',
 ' NINE DRAWER OFFICE TIDY',
 ' OVAL WALL MIRROR DIAMANTE ',
 ' RED SPOT GIFT BAG LARGE',
 ' SET 2 TEA TOWELS I LOVE LONDON ',
 ' SPACEBOY BABY GIFT SET']